# Setup

In [1]:
from proxmox.docs import _proxmox_api_docs
from proxmox.base import ProxmoxAPIChain
from langchain.text_splitter import RecursiveJsonSplitter
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.vectorstores.surrealdb import SurrealDBStore
from langchain_huggingface import HuggingFaceEmbeddings
import nest_asyncio
import os
from dotenv import load_dotenv
from tqdm.autonotebook import tqdm, trange
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate
from langchain_ollama import ChatOllama
from langgraph.graph import END, StateGraph, START
from IPython.display import Image, display
from langfuse.callback import CallbackHandler
from langchain_core.messages import BaseMessage
from typing import List, TypedDict

C:\Users\oumay\AppData\Local\Temp\ipykernel_17444\2933137633.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
load_dotenv()

True

In [3]:
import yaml
import os 
import base64

# Load the YAML file
with open(r'proxmox\tokens.yaml', 'r') as file:
    proxmox_nodes = yaml.safe_load(file)

# Access the dictionary
for node in proxmox_nodes :
    print(f"Name: {node['name']}")
    print(f"Base URL: {node['base_url']}")
    decoded_token = base64.b64decode(os.getenv(node['token'])).decode('utf-8')
    print(f"Token: {decoded_token}\n")
    print(f"Token: {os.getenv(node['token'])}\n")


Name: Proxmox-Node-HCM
Base URL: https://ns31418912.ip-54-38-37.eu:8006/
Token: PVEAPIToken=oumaima@pve!oumaima-token=0cb7eac1-8c3b-4c9c-beec-bf9e35580ccd

Token: UFZFQVBJVG9rZW49b3VtYWltYUBwdmUhb3VtYWltYS10b2tlbj0wY2I3ZWFjMS04YzNiLTRjOWMtYmVlYy1iZjllMzU1ODBjY2Q=

Name: ns31212248
Base URL: https://ns31212248.ip-51-178-74.eu:8006/
Token: PVEAPIToken=oumaima@pve!imx-node5=c3105871-267f-43e5-95dd-663f734745f2

Token: UFZFQVBJVG9rZW49b3VtYWltYUBwdmUhaW14LW5vZGU1PWMzMTA1ODcxLTI2N2YtNDNlNS05NWRkLTY2M2Y3MzQ3NDVmMg==



In [4]:
import os
print(os.getcwd())


c:\Users\oumay\.vscode\agent-scripts\api_chain


In [5]:
surrealdb_url = os.getenv('SURREALDB_URL')
surrealdb_user = os.getenv('SURREALDB_USER')
surrealdb_password = os.getenv('SURREALDB_PWD')

In [6]:
splitter = RecursiveJsonSplitter(max_chunk_size=4000)
docs = splitter.create_documents(_proxmox_api_docs)

In [7]:
# Initialize the SentenceTransformerEmbeddings with trust_remote_code=True
ef = HuggingFaceEmbeddings(
    model_name="infgrad/stella_en_400M_v5",
    model_kwargs={
        "trust_remote_code": True,
    }
)

c:\Users\oumay\.vscode\agent-scripts\venv\Lib\site-packages\xformers\ops\fmha\flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
c:\Users\oumay\.vscode\agent-scripts\venv\Lib\site-packages\xformers\ops\fmha\flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")
Some weights of the model checkpoint at infgrad/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initial

In [8]:
nest_asyncio.apply()

# SurrealDB connection parameters
dburl = "ws://localhost:8000/rpc"
ns = "langchain"
db_name = "proxmox_api_docs"
collection = "proxmox_collection"
db_user = surrealdb_user
db_pass = surrealdb_password

# Initialize the SurrealDBStore with documents
db = SurrealDBStore(
    embedding_function=ef,
    dburl=dburl,
    ns=ns,
    db=db_name,
    collection=collection,
    db_user=db_user,
    db_pass=db_pass
)


# this is needed to initialize the underlying async library for SurrealDB
await db.initialize()

# delete collection from the vectorstore collection if it exists 
await db.adelete(collection = collection)

# add documents to the vectorstore
ids = await db.aadd_documents(docs)

retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Initialize the cross-encoder model
#cross_encoder_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
cross_encoder_model = HuggingFaceCrossEncoder(model_name="cross-encoder/ms-marco-MiniLM-L-6-v2")
compressor = CrossEncoderReranker(model=cross_encoder_model, top_n=3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

# Team approach

In [ ]:
workers = [node["name"] for node in proxmox_nodes]

prompt = PromptTemplate(
    template=("""
    You are a supervisor tasked with routing a query to one of the following workers: {workers}. 
    Given the following user request, respond with the worker to act next. Each worker will perform a task 
    and respond with their results and status. The worker output will be returned to the user. 
    The options for workers are {workers}. Return 'end' if the request isn't related to any of the given workers.

    User request: {question}

    Respond in JSON format with a single key "next" which holds the worker name.

    next :
    """),
    input_variables=["question"],
    partial_variables={"workers": workers}
)

# Initialize the LLM
llm_sup = ChatOllama(model="phi3", temperature=0, format="json")
#llm_sup = ChatOllama(model="llama3.1", temperature=0, format="json")


# Combine prompt and LLM with JsonOutputParser
sup_chain = prompt | llm_sup | JsonOutputParser()


In [ ]:
sup_chain.invoke({
    "question": "How many Vms on node Proxmox-Node-HCM",
})

In [ ]:
class ProxmoxTeamState(TypedDict):
    messages: List[BaseMessage]
    team_members: List[str]
    next: str

In [ ]:
def supervisor(state: ProxmoxTeamState) -> ProxmoxTeamState:
    print(f"processing_request: Current state: {state}")
    question = state["messages"][0]
    result = sup_chain.invoke({"question": question})
    state["next"] = result["next"]
    state["team_members"] = [node["name"] for node in proxmox_nodes]
    return state

In [ ]:
initial_state = {
            "messages": ["list Vms on node 'Proxmox-Node-HCM'"],
            "next": None,
            "team_members": []
        }
supervisor(initial_state)

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize the llm with the new model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0)

In [9]:
from langchain_community.llms import Ollama
from typing import Dict, Any

# Initialize the LLM
#llm = ChatOllama(model="llama3.1", temperature=0, format="json")
llm_kwargs: Dict[str, Any] = {
    "model": "llama3:instruct",
    "temperature": 0,
    "num_ctx": 4096, 
    "format":"json"
}

llm = Ollama(**llm_kwargs)

In [13]:
agents = {}

for node in proxmox_nodes:
    print(node['base_url'])
    decoded_token=base64.b64decode(os.getenv(node['token'])).decode('utf-8')
    chain = ProxmoxAPIChain.from_llm_and_api_docs(
        llm,
        retriever=compression_retriever,
        headers={"Authorization": decoded_token},
        pve_token=decoded_token,
        verbose=True,
        limit_to_domains=[node["base_url"]],
        base_url=node["base_url"],
    )
    agents[node["name"]] = chain

https://ns31418912.ip-54-38-37.eu:8006/
https://ns31212248.ip-51-178-74.eu:8006/


In [14]:
result = agents["Proxmox-Node-HCM"].invoke(("list Vms on node 'Proxmox-Node-HCM'"))
print(result)



> Entering new ProxmoxAPIChain chain...
Retrieved Document 0: {"endpoint": "/api2/json/nodes/{node}/qemu", "method": "get", "summary": "List Virtual machines (VMs) on a Node", "parameters": [{"name": "node", "in": "path", "required": true, "description": "Node ID where VMs are located", "schema": {"type": "string"}}]}
Retrieved Document 1: {"endpoint": "/api2/json/nodes/{node}/qemu/{vmid}/config", "method": "get", "summary": "Get Virtual machine/Vm Configuration (config file)", "parameters": [{"name": "node", "in": "path", "required": true, "description": "Node ID where the VM is located", "schema": {"type": "string"}}, {"name": "vmid", "in": "path", "required": true, "description": "ID of the VM", "schema": {"type": "integer"}}, {"name": "current", "in": "query", "required": false, "description": "Get current values instead of pending values", "schema": {"type": "boolean"}}]}
Retrieved Document 2: {"endpoint": "/api2/json/nodes/{node}/lxc", "methods": "get", "summary": "List LXC Con

In [ ]:
def Worker_node(state: ProxmoxTeamState, node_name: str) -> ProxmoxTeamState:
    print(f"{state['next']} / {node_name}")
    question = state["messages"][0]
    result = agents[node_name].invoke(input=question)
    state["messages"].append(result)
    return state

In [ ]:
state = {
    "next": "Proxmox-Node-HCM",
    "messages": ["How many Vms on node Proxmox-Node-HCM?"]
}

new_state = Worker_node(state, "Proxmox-Node-HCM")
print(new_state)  

In [ ]:
def proxmox_router(state: ProxmoxTeamState) :
    next_worker = state["next"]
    print(f"proxmox_router: Current state: {state}")
    if next_worker in [node["name"] for node in proxmox_nodes]:
        return next_worker
    return "end"

In [ ]:
state = {"next": "Proxmox-Node-HCM"}
result = proxmox_router(state)
print(result) 

In [ ]:
proxmox_graph = StateGraph(ProxmoxTeamState)

for node in proxmox_nodes:
    proxmox_graph.add_node(node["name"], lambda state, node_name=node["name"]: Worker_node(state, node_name))
    proxmox_graph.add_edge(node["name"], END)

proxmox_graph.add_node("supervisor", supervisor)

node_edges = {node["name"]: node["name"] for node in proxmox_nodes}

# Add the "end" key-value pair to the dictionary
node_edges["end"] = END

proxmox_graph.add_conditional_edges(
    "supervisor",
    proxmox_router,
    node_edges,
)

proxmox_graph.add_edge(START, "supervisor")
chain = proxmox_graph.compile()

In [ ]:
display(Image(chain.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
langfuse_handler = CallbackHandler(
    public_key=os.getenv('LANGFUSE_PUBLIC_KEY'),
    secret_key=os.getenv('LANGFUSE_SECRET_KEY'),
    host=os.getenv('LANGFUSE_HOST'),
)

In [ ]:
chain.invoke({
            "messages": ["list Vms on node 'Proxmox-Node-HCM'"],
            "next": None,
            "team_members": []
        }, config={"callbacks": [langfuse_handler]})